<a href="https://colab.research.google.com/github/advik-7/Paradox_Text_classification/blob/main/text_classsifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 print("hello")

hello


In [ ]:
!pip install tensorflow scikit-learn nltk

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Conv1D, GlobalMaxPooling1D, Dropout, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords

In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Conv1D, GlobalMaxPooling1D, Dropout, Dense, Concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords

# Load the training data
train_df = pd.read_csv('/content/train.csv')

# Check column names to ensure 'Label' exists
print("Training Data Columns:", train_df.columns)

# Download stopwords if not already downloaded
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Data preprocessing function
def preprocess_text(text):
    tokens = text.lower().split()  # Basic tokenization and lowercasing
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(tokens)

# Apply preprocessing to the training data
train_df['processed_text'] = train_df['Comment'].apply(preprocess_text)

# Extract statistical features
train_df['text_length'] = train_df['Comment'].apply(lambda x: len(x))
train_df['avg_word_length'] = train_df['Comment'].apply(lambda x: np.mean([len(word) for word in x.split()]) if x.split() else 0)

# Encode labels
label_encoder = LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['Label'])

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_df['processed_text'], train_df['label'], test_size=0.2, random_state=42)

# Vectorization using Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

# Padding sequences
max_length = 100
X_train_pad = pad_sequences(X_train_seq, padding='post', maxlen=max_length)
X_val_pad = pad_sequences(X_val_seq, padding='post', maxlen=max_length)

# Convert additional features to NumPy arrays
X_train_stats = np.array(train_df.loc[X_train.index, ['text_length', 'avg_word_length']])
X_val_stats = np.array(train_df.loc[X_val.index, ['text_length', 'avg_word_length']])

# Build the model
embedding_dim = 128

# Input layers
text_input = Input(shape=(max_length,), name='text_input')
stats_input = Input(shape=(2,), name='stats_input')

# Embedding layer
embedding_layer = Embedding(input_dim=10000, output_dim=embedding_dim, input_length=max_length)(text_input)

# Bi-directional LSTM layer
bi_lstm = Bidirectional(LSTM(128, return_sequences=True))(embedding_layer)

# Convolutional layer
conv_layer = Conv1D(filters=64, kernel_size=1, activation='relu')(bi_lstm)

# Global max pooling
global_max_pooling = GlobalMaxPooling1D()(conv_layer)

# Concatenate with statistical features
concat_layer = Concatenate()([global_max_pooling, stats_input])

# Dropout layers
dropout1 = Dropout(0.1)(concat_layer)
dense_layer = Dense(64, activation='relu')(dropout1)
dropout2 = Dropout(0.1)(dense_layer)

# Output layer
output = Dense(len(label_encoder.classes_), activation='softmax')(dropout2)

# Define the model
model = Model(inputs=[text_input, stats_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with additional features
model.fit([X_train_pad, X_train_stats], y_train, epochs=10, batch_size=32, validation_data=([X_val_pad, X_val_stats], y_val))

# Evaluate the model
y_val_pred = np.argmax(model.predict([X_val_pad, X_val_stats]), axis=-1)

# Calculate accuracy and F1 score
accuracy = accuracy_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred, average='weighted')

print(f"Validation Accuracy: {accuracy:.2f}")
print(f"Validation F1 Score: {f1:.2f}")

Training Data Columns: Index(['Comment', 'Label'], dtype='object')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
5344/5344 ━━━━━━━━━━━━━━━━━━━━ 2119s 396ms/step - accuracy: 0.5640 - loss: 1.1004 - val_accuracy: 0.6880 - val_loss: 0.7335
Epoch 2/10
5344/5344 ━━━━━━━━━━━━━━━━━━━━ 2096s 391ms/step - accuracy: 0.6906 - loss: 0.7257 - val_accuracy: 0.6955 - val_loss: 0.7047
Epoch 3/10
5344/5344 ━━━━━━━━━━━━━━━━━━━━ 2114s 389ms/step - accuracy: 0.7049 - loss: 0.7101 - val_accuracy: 0.6967 - val_loss: 0.6947
Epoch 4/10
5344/5344 ━━━━━━━━━━━━━━━━━━━━ 2072s 388ms/step - accuracy: 0.7107 - loss: 0.6675 - val_accuracy: 0.6969 - val_loss: 0.7030
Epoch 5/10
5344/5344 ━━━━━━━━━━━━━━━━━━━━ 2107s 385ms/step - accuracy: 0.7230 - loss: 0.6409 - val_accuracy: 0.6949 - val_loss: 0.7106
Epoch 6/10
5344/5344 ━━━━━━━━━━━━━━━━━━━━ 1992s 372ms/step - accuracy: 0.7327 - loss: 0.6179 - val_accuracy: 0.6928 - val_loss: 0.7216
Epoch 7/10
5344/5344 ━━━━━━━━━━━━━━━━━━━━ 2019s 375ms/step - accuracy: 0.7427 - loss: 0.5961 - val_accuracy: 0.6916 - val_loss: 0.7300
Epoch 8/10
5344/5344 ━━━━━━━━━━━━━━━━━━━━ 2031s 380ms/s

In [ ]:
test="/content/test.csv"

In [ ]:
# Load the test data
test_df = pd.read_csv('/content/test.csv')

# Apply preprocessing function to the test data
test_df['processed_text'] = test_df['Comment'].apply(preprocess_text)

# Extract statistical features for the test data
test_df['text_length'] = test_df['Comment'].apply(lambda x: len(x))
test_df['avg_word_length'] = test_df['Comment'].apply(lambda x: np.mean([len(word) for word in x.split()]) if x.split() else 0)

# Transform text data into sequences and pad them
X_test_seq = tokenizer.texts_to_sequences(test_df['processed_text'])
X_test_pad = pad_sequences(X_test_seq, padding='post', maxlen=max_length)

# Convert additional features to NumPy arrays
X_test_stats = np.array(test_df[['text_length', 'avg_word_length']])


In [ ]:
# Predict on the test data
y_test_pred = np.argmax(model.predict([X_test_pad, X_test_stats]), axis=-1)

# Convert predictions back to original labels
y_test_labels = label_encoder.inverse_transform(y_test_pred)

# Add predictions to the test DataFrame
test_df['Predicted_Label'] = y_test_labels


2863/2863 ━━━━━━━━━━━━━━━━━━━━ 334s 117ms/step


In [ ]:
# Save predictions to a new CSV file
output_csv_path = '/content/test_predictions.csv'
test_df[[ 'Predicted_Label']].to_csv(output_csv_path, index=False)

print(f"Predictions saved to {output_csv_path}")


Predictions saved to /content/test_predictions.csv


In [ ]:
test_df.head()

,ID,Comment,processed_text,text_length,avg_word_length,Predicted_Label
0,0,bhai ko kiska paper ha,bhai ko kiska paper ha,23,3.600000,irrelevant
1,1,thank me later,thank later,14,4.000000,irrelevant
2,2,thankqqqqq mam for this video,thankqqqqq mam video,29,5.000000,feedback
3,3,apki teaching se physics sahi main ek visuliza...,apki teaching se physics sahi main ek visuliza...,76,6.000000,feedback
4,4,i didnt understand the differentiation part co...,didnt understand differentiation part could an...,68,5.272727,doubt
